In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
!pip install accelerate

In [ ]:
 pip install bitsandbytes

In [1]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", load_in_8bit=True, device_map={"": 0}, torch_dtype=torch.float16
)  # doctest: +IGNORE_RESULT

url = "https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small"
image = Image.open(requests.get(url, stream=True).raw)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


pytorch_model.bin.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
img_url = 'https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?format=jpg&name=small' 
raw_image = Image.open(requests.get(img_url, stream=True).raw)

prompt = "Question: What is the picture showing? Answer:"
inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

In [ ]:
inputs = processor(images=raw_image, return_tensors="pt").to(device, torch.float16)

generated_ids = model.generate(**inputs,max_new_tokens = 1024)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

In [3]:
import pandas as pd
import numpy as np
data = pd.read_excel('/kaggle/input/behavior/behaviour_simulation_train.xlsx')
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)

In [4]:
df_new = df[14545:150001]

In [5]:
df_new

,id,date,likes,content,username,media,inferred company
14545,14546,2019-04-05 22:30:00,21,"#FBF to one of our earliest wireless routers, the #ASUS WL-500g Premium. #FlashbackFriday <hyperlink>",ASUSUSA,"[Photo(previewUrl='https://pbs.twimg.com/media/D3Z9qMaUEAADXge?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/D3Z9qMaUEAADXge?format=jpg&name=large')]",asus
14546,14547,2020-03-02 10:02:03,1,Covid -19: Kano Drags Airlines To NCAA For Violation Of Safety Rules <hyperlink> <hyperlink>,IndependentNGR,"[Photo(previewUrl='https://pbs.twimg.com/media/ESGJQQJUYAAlRbZ?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/ESGJQQJUYAAlRbZ?format=jpg&name=large')]",independent
14547,14548,2019-04-11 09:45:00,129,Join the #PrepQuiz for #BTSonInsider with <mention> now and get ready to win big! <hyperlink>,paytminsider,"[Video(thumbnailUrl='https://pbs.twimg.com/ext_tw_video_thumb/1116236937394839559/pu/img/EdH9BE8ORY1TM4BU.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1116236937394839559/pu/vid/720x720/blj9nosrxpHiF27j.mp4?tag=8', bitrate=1280000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1116236937394839559/pu/vid/320x320/JF-zStZ_urZbycwR.mp4?tag=8', bitrate=432000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/ext_tw_video/1116236937394839559/pu/vid/480x480/DhbeJNplGsXwCN5Z.mp4?tag=8', bitrate=832000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/ext_tw_video/1116236937394839559/pu/pl/XiPrVZsW1ZZ5eCuK.m3u8?tag=8', bitrate=None)], duration=31.28, views=740)]",paytm
14548,14549,2020-04-04 17:42:14,11901,"Who wants a slice of cake? 🎂 \n\nOn this day in 1975, Microsoft was born. <hyperlink>",Microsoft,"[Photo(previewUrl='https://pbs.twimg.com/media/EUxvCjiUYAE8CSA?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/EUxvCjiUYAE8CSA?format=jpg&name=large')]",microsoft
14549,14550,2018-10-24 16:55:02,1,Great #EUCyberSummit today talking to customers about using threat intelligence to stay more secure. For more expert cyber advice 👉 <hyperlink> <hyperlink>,CiscoUKI,"[Photo(previewUrl='https://pbs.twimg.com/media/DqScza6WoAAYuL9?format=jpg&name=small', fullUrl='https://pbs.twimg.com/media/DqScza6WoAAYuL9?format=jpg&name=large')]",cisco
...,...,...,...,...,...,...,...
149996,149997,2019-05-26 18:48:15,1298,It's been a pleasure #MonacoGP 🇲🇨 We'll catch you on the streets next year 👊 #F1 <hyperlink>,redbullracing,"[Video(thumbnailUrl='https://pbs.twimg.com/media/D7g6rgmWwAAgwJS.jpg', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1132564096468045824/vid/640x360/6EiSwTzdBfP3JNgr.mp4?tag=12', bitrate=832000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1132564096468045824/vid/320x180/yD-3q_cRGuS1_smY.mp4?tag=12', bitrate=288000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1132564096468045824/pl/r-DV9WIX31Jguorx.m3u8?tag=12', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1132564096468045824/vid/1280x720/nOA4J3ely53Z4qXo.mp4?tag=12', bitrate=2176000)], duration=7.0, views=39668)]",oracle
149997,149998,2020-01-30 16:57:17,93,"""Everything is on the table"" - <mention> on the #2 pick in the draft #PatMcAfeeShowLIVE <hyperlink>",PatMcAfeeShow,"[Video(thumbnailUrl='https://pbs.twimg.com/amplify_video_thumb/1222926325188567042/img/1ZSq1ry2Sj4Ibbtv.jpg', variants=[VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1222926325188567042/pl/f0P1fAbRdu8LNP2G.m3u8?tag=13', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1222926325188567042/vid/640x360/UBxzYL-IIb-H6ZMp.mp4?tag=13', bitrate=832000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1222926325188567042/vid/1280x720/moMu57VBBHQ5GsUd.m

In [6]:
import re

def extract_url(text):
    """
    Extracts the URL from a given text.
    """
    pattern = r"(https?://[^\s]+large)"
    video_thumbnail_pattern = r"thumbnailUrl='(https?://[^']+)'"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        video_match = re.search(video_thumbnail_pattern, text)
        if video_match:
            return video_match.group(1)
    return None


In [ ]:
extract_url("[Video(thumbnailUrl='https://pbs.twimg.com/media/Ecz7MrQWoAIJ1u-.png', variants=[VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1282676157499015168/vid/720x720/oJ68bJ_syS_wk-_G.mp4?tag=13', bitrate=1280000), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1282676157499015168/vid/320x320/ulX5aS6ovZU-skcg.mp4?tag=13', bitrate=432000), VideoVariant(contentType='application/x-mpegURL', url='https://video.twimg.com/amplify_video/1282676157499015168/pl/RCnrYWRZFOWYTnp0.m3u8?tag=13', bitrate=None), VideoVariant(contentType='video/mp4', url='https://video.twimg.com/amplify_video/1282676157499015168/vid/480x480/0o9HCbMI0gWlzf0J.mp4?tag=13', bitrate=832000)], duration=6.039, views=3826)]")

In [7]:
urls = []
for text in df_new["media"]:
  url = extract_url(text)
  urls.append(url)

In [ ]:
df_new.loc[:, "urls"] = urls

In [ ]:
cnt_none = df_new['urls'].isnull().sum()
cnt_none

In [ ]:
df_new.head(10)

In [8]:
from tqdm import tqdm
import csv
captions = []

for url in tqdm(urls):
  if url:
    try:
      raw_image = Image.open(requests.get(url, stream=True).raw)
      prompt = "Question: What is the picture showing? Answer:"
      inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(device="cuda", dtype=torch.float16)
      generated_ids = model.generate(**inputs)
      generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
      captions.append("Picture of " + generated_text)
        
      with open("/kaggle/working/captions.csv", "a", newline="") as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=["caption"])
          writer.writerow({"caption": "Picture of " + generated_text})
    except Exception as e:
      print(f"Error generating caption for URL: {url}")
      print(e)
      captions.append(None)
      with open("/kaggle/working/captions.csv", "a", newline="") as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=["caption"])
          writer.writerow({"caption": "None"})
  else:
    captions.append(None)
    with open("/kaggle/working/captions.csv", "a", newline="") as csvfile:
          writer = csv.DictWriter(csvfile, fieldnames=["caption"])
          writer.writerow({"caption": "None"})

  0%|          | 0/135456 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 86/135456 [03:22<65:19:19,  1.74s/it] 

Error generating caption for URL: https://pbs.twimg.com/media/EG8BqZtWkAA5YQR?format=jpg&name=large
cannot identify image file <_io.BytesIO object at 0x7a1bc08c0c70>


  0%|          | 152/135456 [05:50<72:54:49,  1.94s/it] 

Error generating caption for URL: https://pbs.twimg.com/media/DcUATotX0AIhl3D?format=jpg&name=large
cannot identify image file <_io.BytesIO object at 0x7a1bc09c1940>


  0%|          | 314/135456 [12:02<63:29:32,  1.69s/it] 

...

Error generating caption for URL: https://pbs.twimg.com/media/EJFnkgWXkAAj0Py?format=jpg&name=large
cannot identify image file <_io.BytesIO object at 0x7a1bc0ce0590>


  9%|▉         | 12575/135456 [8:05:09<63:55:40,  1.87s/it] 

Error generating caption for URL: https://pbs.twimg.com/media/EqSAvCqXEAECBwz?format=jpg&name=large
cannot identify image file <_io.BytesIO object at 0x7a1bc07571a0>


  9%|▉         | 12579/135456 [8:05:20<79:00:56,  2.31s/it]


KeyboardInterrupt: 

In [ ]:
df_new[1:14]

In [ ]:
urls_fr_dwnld = []
for text in df["media"]:
  url = extract_url(text)
  urls_fr_dwnld.append(url)

In [ ]:
urls_fr_dwnld

In [ ]:
import requests
import os

def download_images(urls):
    if not os.path.exists("/kaggle/working/images"):
        os.makedirs("/kaggle/working/images")

    for index, url in enumerate(urls, start=1):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                file_name = f"/kaggle/working/images/{index}.jpg"
                with open(file_name, 'wb') as file:
                    file.write(response.content)
                print(f"Downloaded image {index}")
            else:
                print(f"Failed to download image {index} from {url}")
        except Exception as e:
            print(f"Error downloading image {index}: {str(e)}")
            continue

In [ ]:
download_images(urls_fr_dwnld)